IMPORTS

In [1]:
import numpy as np
import pandas as pd
import joblib

In [2]:
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

DATAFRAME

In [3]:
df = pd.read_csv('/home/mamisoa/ITU/L3/Mr_Tsinjo/Sardinas_patterson/python/codes.csv')
df = df.drop(['language'], axis=1)
df

,length,average_word_length,frequency_of_1,frequency_of_0,frequency_of_01,frequency_of_10,frequency_of_11,frequency_of_00,nc,levenshtein,special,classe
0,3,4.00,2.250000,0.750000,0.750000,0.250000,0.750000,0.000000,9.0,4.333333,5.823142e-08,True
1,2,3.00,1.000000,1.000000,0.333333,0.333333,0.333333,0.333333,4.0,4.000000,9.150652e-08,True
2,2,3.50,0.571429,1.428571,0.000000,0.285714,0.000000,0.571429,4.0,5.000000,2.578820e-07,True
3,4,4.25,1.882353,2.117647,0.705882,0.705882,0.470588,0.705882,16.0,4.500000,2.402150e-05,True
4,2,5.50,1.090909,0.909091,0.181818,0.545455,0.363636,0.181818,4.0,3.000000,2.370851e-06,True
...,...,...,...,...,...,...,...,...,...,...,...,...
4995,3,5.00,2.200000,0.800000,0.200000,0.200000,1.000000,0.400000,9.0,3.666667,3.743449e-08,False
4996,5,2.80,2.857143,2.142857,1.071429,1.071429,0.714286,0.357143,25.0,1.700000,8.636552e-06,False
4997,10,4.80,3.750000,6.250000,1.250000,2.083333,1.250000,2.083333,100.0,3.266667,7.997897e+04,False
4998,6,4.17,2.158273,3.836930,1.199041,1.199041,0.239808,1.199041,36.0,3.266667,1.885160e-02,False


TRAINING WITH RANDOM FOREST

In [4]:
X = df.iloc[:, 0:-1].values
Y = df.iloc[:, -1].values

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25,random_state = 0)

In [6]:
clf = RandomForestClassifier(random_state=0)
clf = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [7]:
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[585  69]
 [ 57 539]]


In [8]:
accuracy_score(y_test, y_pred)

0.8992

IMPORTATION MODELE

In [9]:
joblib.dump(clf , "/home/mamisoa/ITU/L3/Mr_Tsinjo/Sardinas_patterson/modele/code.joblib")

['/home/mamisoa/ITU/L3/Mr_Tsinjo/Sardinas_patterson/modele/code.joblib']

In [2]:
# modele = joblib.load("/home/mamisoa/ITU/L3/Mr_Tsinjo/Sardinas_patterson/modele/code.joblib")
modele = joblib.load("/home/mamisoa/ITU/L3/Mr_Tsinjo/Sardinas_patterson/modele/code_ok_1.joblib")

IMPLEMENTATION SARDINAS-PATTERSON ALGORITHM

In [3]:
class SardinasPatterson:
    language = list()
    step = list()
    epsilon = ""
    
    def __init__(self, language):
        self.language = language
        self.step.clear()

    def divide(self, L1, L2):
        res = set()
        for elt in L1:
            for item in L2:
                if item.startswith(elt):
                    if(item[len(elt):] == ''):
                        self.epsilon = elt
                    res.add(item[len(elt):])
        return res
    
    def get_L1(self):
        res = self.divide(self.language, self.language)
        res.remove('')
        return res
    
    def get_L_n_plus_1(self, Ln):
        res = self.divide(Ln, self.language)
        return res.union(self.divide(self.language, Ln))
    
    def get_mot(self, lst, mot):
        for item in self.language:
            for elt in lst:
                if(item == (elt + mot)):
                    return elt
        return mot

    def get_contre_exemple(self):
        res = []
        mot = self.epsilon
        temp = self.step
        res.append(mot)
        for i in range(len(temp) - 2, -1, -1 ):
            mot = self.get_mot(temp[i-1], mot)
            res.append(mot)
        return ''.join(res)

    def make_code(self):
        initial = self.language.copy()
        temp = initial.copy()
        temp_2 = None
        for i in range(len(temp)):
            self.language = initial.copy()
            temp = initial.copy()
            temp.pop(i)
            self.language = temp
            temp_2 = temp.copy()
            if(self.is_code()[0] == True):
                return self.language
            for j in range(len(temp_2)):
                temp_2.pop(j)
                self.language = temp_2
                if(self.is_code()[0] == True):
                    return self.language
        return list()

    def is_code(self):
        if(len(self.language) == 1):
            return True, 'It stops at L1'
        temp = self.get_L1()
        count = 1
        self.step.append(temp)
        while('' not in temp):
            temp = self.get_L_n_plus_1(temp)
            if temp in self.step:
                return True, 'It stops at L' + str(count) 
            self.step.append(temp)
            count += 1
        self.step.append(self.language)
        return False, 'It stops at L' + str(count)

TREATMENT OF THE CODE SENT

In [4]:
import math
from nltk.metrics.distance import edit_distance

def get_average_word_length(language):
    res = 0
    for elt in language:
        res += len(elt)
    return round((res / len(language)), 2)

def get_percent_of_elt(language, letter):
    res = 0
    for elt in language:
        res += elt.count(letter) / len(language)
    return res

def get_number_of_elt(language, letter):
    res = 0
    for elt in language:
        res += elt.count(letter)
    return res

def get_frequency_of_number(language, letter):
    res = 0
    diviseur = get_average_word_length(language)
    for elt in language:
        res += elt.count(letter) / diviseur
    return res

def letter_probability(string, letter):
    return string.count(letter) / len(string)
    

def word_entropy(string):
    p_0 = letter_probability(string, '0')
    p_1 = letter_probability(string, '1')
    log_0 = 0
    if(p_0 != 0):
        log_0 = math.log(p_0, 2)
    log_1 = 0
    if(p_1 != 0):
        log_1 = math.log(p_1, 2)
    res = -1
    res *= ((p_0 * log_0) + (p_1 * log_1))
    return res

def entropy(language):
    res = 0
    for elt in language:
        res += word_entropy(elt)
    return res/len(language) 


def calculate_nc(language):
    res = 0
    for string in language:
        p_0 = letter_probability(string, '0')
        p_1 = letter_probability(string, '1')
        res += (p_0 * len(language)) + (p_1 * len(language))
    return res

def count_word_composed_of_letter(language, letter):
    occ = 0
    for elt in language:
        if(elt.count(letter) == len(elt)):
            occ += 1
    return occ

def count_word_composed_of_mixed_letter(language):
    occ = 0
    for elt in language:
        if('01' in elt or '10' in elt):
            occ += 1
    return occ

def levenshtein_average_interval(language):
    total_interval = 0
    pair_nb = 0
    for i in range(len(language)):
        for j in range(i+1, len(language)):
            total_interval += edit_distance(language[i], language[j])
            pair_nb += 1
    return total_interval / pair_nb if pair_nb > 0 else 0


def calcul_value(language):
    res = 0
    occ_0 = count_word_composed_of_letter(language, '0')
    occ_1 = count_word_composed_of_letter(language, '1')
    occ_mixte = count_word_composed_of_mixed_letter(language)
    res = (occ_0 * 2) + (occ_1 * 2) + (occ_mixte * 1) 
    return res / 5

def parity_calculation(language):
    res = 0
    for elt in language:
        res += elt.count('1')
    return res % 2


def special(word):
    value = "".join(word)
    number = int(value, 2)
    return number / (len(word)*240420020)

    
def get_language_data(lang):
    value = {}
    value['length'] = (str(len(lang)))
    value['average_word_length'] = str(get_average_word_length(lang))
#     value['percentage_of_0'] = str(get_percent_of_elt(lang, '0'))
#     value['percentage_of_1'] = str(get_percent_of_elt(lang, '1'))
#     value['percentage_of_01'] = str(get_percent_of_elt(lang, '01'))
#     value['percentage_of_10'] = str(get_percent_of_elt(lang, '10'))
#     value['percentage_of_00'] = str(get_percent_of_elt(lang, '00'))
#     value['percentage_of_11'] = str(get_percent_of_elt(lang, '11'))
    value['frequency_of_0'] = str(get_frequency_of_number(lang, '0'))
    value['frequency_of_1'] = str(get_frequency_of_number(lang, '1'))
    value['frequency_of_01'] = str(get_frequency_of_number(lang, '01'))
    value['frequency_of_10'] = str(get_frequency_of_number(lang, '10'))
    value['frequency_of_00'] = str(get_frequency_of_number(lang, '00'))
    value['frequency_of_11'] = str(get_frequency_of_number(lang, '11'))
#     value['levenshtein'] = str(levenshtein_average_interval(lang))
#     value['nc'] = str(calculate_nc(lang))
    value['entropy'] = str(entropy(lang))
#     value['parity'] = str(parity_calculation(lang))
    value['special'] = str(special(lang))
    return value

SERVER-SIDE powered by Flask

In [ ]:
from flask import Flask,request, jsonify
from flask_cors import CORS, cross_origin

app = Flask(__name__)

    
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'

@app.route('/', methods=['GET'])
@cross_origin()
def home():
    return "<h1>Index</h1><p>"


@app.route('/is_code', methods = ['POST' , 'GET'])
@cross_origin()
def is_code():
    try:
        code = request.form.get('code').replace(" ", "")
        code = code.split(",")
        sp = SardinasPatterson(code)
    #     sp.language = code
        temp = get_language_data(code)
        value = pd.DataFrame(temp, index=[0])
        sp_result = sp.is_code()
        model_result = modele.predict(value)[0]
        return jsonify({"sp_result": str(sp_result[0]), "model_result": str(model_result), "code": code})
    except Exception as e:
        print(e)
        return jsonify({"sp_result": "", "model_result": "", "code": str(e)})
    
if __name__ == '__main__':
    app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
/usr/lib/python3/dist-packages/sklearn/base.py:413: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
127.0.0.1 - - [28/May/2024 15:33:57] "POST /is_code HTTP/1.1" 200 -
/usr/lib/python3/dist-packages/sklearn/base.py:413: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
127.0.0.1 - - [28/May/2024 15:34:15] "POST /is_code HTTP/1.1" 200 -
/usr/lib/python3/dist-packages/sklearn/base.py:413: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
127.0.0.1 - - [28/May/2024 15:36:00] "POST /is_code HTTP/1.1" 200 -
/usr/lib/python3/dist-packages/sklearn/base.py:413: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
127.0.0.1 - - [28/May/2024 15:36:03] "POST /is_code HTTP/1.1" 200 -
/usr/lib/python3/di